In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
from geodatasets import get_path
from function_06 import load_data_with_delimiters
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap


In [2]:
file_info = "../data/raw/ongevallen.txt"

df_acc2 =pd.read_csv(file_info, low_memory=False)

In [5]:
df_acc2.shape, df_acc2.isnull().sum(), df_acc2.columns

((1193484, 66),
 VKL_NUMMER          0
 REGNUMMER      197529
 PVOPGEM       1166161
 DATUM_VKL     1193484
 DAG_CODE      1193484
                ...   
 DISTRCODE      964941
 DISTRNAAM      964941
 DAGTYPE       1193484
 IND_ALC       1193484
 WEEKNR        1193484
 Length: 66, dtype: int64,
 Index(['VKL_NUMMER', 'REGNUMMER', 'PVOPGEM', 'DATUM_VKL', 'DAG_CODE',
        'MND_NUMMER', 'JAAR_VKL', 'TIJDSTIP', 'UUR', 'DDL_ID', 'AP3_CODE',
        'AP4_CODE', 'AP5_CODE', 'ANTL_SLA', 'ANTL_DOD', 'ANTL_GZH', 'ANTL_SEH',
        'ANTL_GOV', 'ANTL_PTJ', 'ANTL_TDT', 'MNE_CODE', 'AOL_ID', 'NIVEAUKOP',
        'WSE_ID', 'WSE_AN', 'BEBKOM', 'MAXSNELHD', 'WVL_ID', 'WVG_ID', 'WVG_AN',
        'WDK_ID', 'WDK_AN', 'LGD_ID', 'ZAD_ID', 'WGD_CODE_1', 'WGD_CODE_2',
        'BZD_ID_VM1', 'BZD_ID_VM2', 'BZD_ID_VM3', 'BZD_VM_AN', 'BZD_ID_IF1',
        'BZD_ID_IF2', 'BZD_ID_IF3', 'BZD_IF_AN', 'BZD_ID_TA1', 'BZD_ID_TA2',
        'BZD_ID_TA3', 'BZD_TA_AN', 'JTE_ID', 'WVK_ID', 'HECTOMETER', 'FK_VELD5',
       

In [7]:
relevant_columns = [
    'VKL_NUMMER', 'TIJDSTIP', 'UUR', 'JAAR_VKL',
    'ANTL_DOD', 'ANTL_GZH', 'ANTL_SEH',
    'WVK_ID', 'GME_NAAM', 'PLT_NAAM',
    'BZD_VM_AN', 'BZD_IF_AN', 'BZD_TA_AN'
]

# Keep only those columns
df_acc2 = df_acc2[relevant_columns]

# Drop rows where critical identifiers or keys are missing
df_acc2 = df_acc2.dropna(subset=['VKL_NUMMER', 'WVK_ID'])


In [9]:
 df_acc2.shape

(864510, 13)

In [11]:
file_info = "../data/raw/partijen.txt"

df_A=pd.read_csv(file_info, low_memory=False)

In [ ]:
df_A.head()

In [ ]:
df_A['PTJ_ID'].head()

In [ ]:
df_A.isnull().sum()

In [ ]:
# df_A = df_A.dropna(subset=['TDT_AN'])
# df_A = df_A.dropna(subset=['OTE_AN'] )

In [ ]:
df_A[df_A['OTE_AN'].str.contains("|")==True][['OTE_AN']].head()

In [13]:
df_A['OTE_AN'].unique()

array([nan, 'Verloren onderdeel', 'Verloren lading', 'politievoertuig',
       'Geparkeerde Aanhangwagen', 'politievtg opt en geluidsignal',
       'Onderdeel botsing', 'kampeerwagen', 'ambu opt en geluidsignalen',
       'politiemotor', 'politievrtg opt', 'quad', 'E-bike', 'politievrtg',
       'bouwkraan', 'hijskraan', 'Paard en wagen', 'kolkwagen',
       'graafmachine', 'hakselaar', 'OPLEGGER', 'Onbekend', 'AANHANGER',
       'BOMEN', 'AANHANGER < 750', 'HIJSKRAAN', 'HUISDIER', 'BOUWKRAAN',
       'LANDBOUWVOERTU', 'COMPUTER', 'STRUIKEN', 'RIJTUIG',
       'BOUWVOERTUIG', 'KENTEKENBEWIJS', 'WILD', 'KAMPEERWAGEN',
       'OPENBAARVERVOE', 'TOURCARAVAN', 'OVERIG', 'OPSLAGTANK',
       'GRAAFMACHINE', 'CONTAINER', 'VEE', 'RIJBEWIJS', 'QUAD',
       'VUURWAPEN', 'ROLLATOR', 'HOOGWERKER', 'KENTEKENPLAAT', 'GRILLE',
       'VERLICHTING', 'VORKHEFTRUCK', 'NAVIGATIESYSTE', 'WINKELWAGEN',
       'ROLSTOEL', 'PLANTEN', 'VEEGMACHINE', 'TUINGEREEDSCHA', 'WALS',
       'MINIBIKE', 'KLANTENPAS',

In [ ]:
df_A['TDT_AN'].unique()

In [19]:
# Keywords related to bikes in OTE_AN
bike_keywords_ote = [
    'FIETS', 'E-BIKE', 'SPEED PEDELEC', 'SPEEDPEDELEC', 'FIETSKAR',
    'FIETSAANHANGER', 'DRIEWIELER', 'HANDBIKE', 'FATBIKE', 'FAT BIKE',
    'SKELTER', 'BUGGY', 'ELEKTRISCHE-STEP', 'ELEKTRISCHE STEP', 'ELEKTRISCHE DRIEWIELER'
]

# Keywords indicating cyclist-related circumstances in TDT_AN
bike_keywords_tdt = [
    'FIETS', 'FIETSPAD', 'EVENWICHT', 'PORTIER OPENEN', 'GEEN VERLICHTING',
    'OBSTAKEL', 'SLECHT WEGDEK', 'AUTO OP FIETSPAD', 'ONVOORZICHTIG OP WEG'
]

bike_accidents = df_A[(df_A['OTE_AN'].isin(bike_keywords_ote)) | (df_A['TDT_AN'].isin(bike_keywords_tdt))]

# Filter rows where either column contains one of the keywords (case-insensitive)
#bike_accidents = df_A[
#    df_A['OTE_AN'].str.contains('|'.join(bike_keywords_ote), case=False, na=False) |
#    df_A['TDT_AN'].str.contains('|'.join(bike_keywords_tdt), case=False, na=False)
#]


In [21]:
bike_accidents = bike_accidents[['PTJ_ID', 'VKL_NUMMER', 'NUMMER','OTE_AN','TDT_AN']]


In [23]:
bike_accidents[['PTJ_ID']].head()

,PTJ_ID
32328,26052294
60168,26084674
66861,26093279
74127,26083750
77081,26061609


In [25]:
bike_accidents.isnull().sum()

PTJ_ID          0
VKL_NUMMER      0
NUMMER          0
OTE_AN          8
TDT_AN        387
dtype: int64

In [27]:
file_info = "../data/raw/wegvakgeografie_01012024.csv"

wegvak_df=pd.read_csv(file_info, low_memory=False)

In [29]:
# Filter the wegvak dataset for rows where the municipality is Amsterdam
wegvak_amsterdam = wegvak_df[wegvak_df['gme_naam'].str.lower() == 'amsterdam']

# Check how many rows and preview a few
wegvak_amsterdam.shape, wegvak_amsterdam[['wvk_id', 'gme_naam', 'shape']].head()


((38887, 42),
         wvk_id   gme_naam                                              shape
 10   239374087  Amsterdam  LINESTRING (119808.14400000125 487129.29500000...
 31   600416574  Amsterdam  LINESTRING (120287.60770000145 485849.17119999...
 258  234376081  Amsterdam  LINESTRING (117476.56010000035 488051.62629999...
 264  231369048  Amsterdam  LINESTRING (115864.78000000119 484508.03849999...
 265  230374012  Amsterdam  LINESTRING (115203.02639999986 487453.90980000...)

In [31]:
columns_to_keep = [
    'wvk_id',           # Road Segment ID
    'wvk_begdat',       # Start Date
    'wvk_enddat',       # End Date (optional if not always available)
    'wegnummer',        # Road Number
    'stt_naam',         # Street Name
    'wpsnaam',          # Neighborhood Name
    'gme_naam',         # Municipality Name
    'rijrichtng',       # Driving Direction
    'wegbehsrt',        # Road Type / Category
    'shape'             # Geometry
]


In [33]:
wegvak_amsterdam = wegvak_amsterdam[columns_to_keep]

# Drop rows with NaN in critical columns
wegvak_amsterdam = wegvak_amsterdam.dropna(subset=['wvk_id', 'wegnummer', 'stt_naam', 'shape'])


In [35]:
# Ensure column name consistency for the join
bike_data_geo = df_acc2.copy()

# Confirm WVK_ID exists in both datasets
if 'WVK_ID' in bike_data_geo.columns and 'wvk_id' in wegvak_amsterdam.columns:
    # Merge on WVK_ID
    bike_amsterdam_geo = bike_data_geo.merge(
        wegvak_amsterdam,
        left_on='WVK_ID',
        right_on='wvk_id',
        how='inner'
    )
    match_found = True
else:
    bike_amsterdam_geo = pd.DataFrame()
    match_found = False

# Output results
# match_found, bike_amsterdam_geo[['Accident_ID', 'Vehicle_Type', 'Municipality', 'wvk_id', 'shape']].head() if match_found else "No matches found"


In [37]:
bike_amsterdam_geo.columns

Index(['VKL_NUMMER', 'TIJDSTIP', 'UUR', 'JAAR_VKL', 'ANTL_DOD', 'ANTL_GZH',
       'ANTL_SEH', 'WVK_ID', 'GME_NAAM', 'PLT_NAAM', 'BZD_VM_AN', 'BZD_IF_AN',
       'BZD_TA_AN', 'wvk_id', 'wvk_begdat', 'wvk_enddat', 'wegnummer',
       'stt_naam', 'wpsnaam', 'gme_naam', 'rijrichtng', 'wegbehsrt', 'shape'],
      dtype='object')

In [39]:
bike_accidents.isnull().sum()

PTJ_ID          0
VKL_NUMMER      0
NUMMER          0
OTE_AN          8
TDT_AN        387
dtype: int64

In [41]:
bike_accidents.head()

,PTJ_ID,VKL_NUMMER,NUMMER,OTE_AN,TDT_AN
32328,26052294,20140096088,1,NaN,PORTIER OPENEN
60168,26084674,20140113085,3,FIETSAANHANGER,NaN
66861,26093279,20140117493,2,SKELTER,NaN
74127,26083750,20140112604,1,NaN,PORTIER OPENEN
77081,26061609,20140101008,3,FIETSAANHANGER,NaN


In [43]:
bike_data = bike_accidents.merge(bike_amsterdam_geo, on='VKL_NUMMER', how='outer')


In [49]:
bike_data.shape

(12309, 27)

In [47]:
bike_data.isnull().sum()

PTJ_ID        11914
VKL_NUMMER        0
NUMMER        11914
OTE_AN        11922
TDT_AN        12301
TIJDSTIP      12309
UUR           12309
JAAR_VKL        395
ANTL_DOD      12309
ANTL_GZH      12309
ANTL_SEH      12309
WVK_ID          395
GME_NAAM        395
PLT_NAAM        395
BZD_VM_AN     12304
BZD_IF_AN     11873
BZD_TA_AN     12239
wvk_id          395
wvk_begdat      395
wvk_enddat     7667
wegnummer       395
stt_naam        395
wpsnaam         395
gme_naam        395
rijrichtng      395
wegbehsrt       395
shape           395
dtype: int64

In [51]:
# Drop 'WVK_ID' (duplicate of 'wvk_id') and 'Unnamed: 0' (auto index column)
bike_data = bike_data.drop(columns=['WVK_ID'])
rename_dict = {
    'PTJ_ID': 'Participant_ID',
    'VKL_NUMMER': 'Accident_ID',
    'NUMMER': 'Case_Number',
    'OTE_AN': 'Object_Involved',
    'TDT_AN': 'Contributing_Factor',
    'TIJDSTIP': 'Time',
    'UUR': 'Hour',
    'JAAR_VKL': 'Accident_Year',
    'ANTL_DOD': 'Fatalities',
    'ANTL_GZH': 'Severe_Injuries',
    'ANTL_SEH': 'Emergency_Room_Injuries',
    'WVK_ID': 'RoadSegment_ID',
    'GME_NAAM': 'Municipality',
    'PLT_NAAM': 'Police_Area',
    'BZD_VM_AN': 'Vehicle_Type',
    'BZD_IF_AN': 'Object_Collided_With',
    'BZD_TA_AN': 'Other_Party_Involved',
    'wvk_id': 'RoadSegment_ID',  # duplicate
    'wvk_begdat': 'Segment_StartDate',
    'wvk_enddat': 'Segment_EndDate',
    'wegnummer': 'RoadNumber',
    'stt_naam': 'StreetName',
    'wpsnaam': 'Neighborhood',
    'gme_naam': 'Municipality',  # duplicate
    'rijrichtng': 'DrivingDirection',
    'wegbehsrt': 'RoadType',
    'shape': 'Geometry'
}

# Step 3: Optional rename for clarity (if needed)
bike_data = bike_data.rename(columns=rename_dict)

In [53]:
# Display updated column names
bike_data.columns.tolist()

['Participant_ID',
 'Accident_ID',
 'Case_Number',
 'Object_Involved',
 'Contributing_Factor',
 'Time',
 'Hour',
 'Accident_Year',
 'Fatalities',
 'Severe_Injuries',
 'Emergency_Room_Injuries',
 'Municipality',
 'Police_Area',
 'Vehicle_Type',
 'Object_Collided_With',
 'Other_Party_Involved',
 'RoadSegment_ID',
 'Segment_StartDate',
 'Segment_EndDate',
 'RoadNumber',
 'StreetName',
 'Neighborhood',
 'Municipality',
 'DrivingDirection',
 'RoadType',
 'Geometry']

In [55]:
bike_data.columns = bike_data.columns.str.lower()


In [57]:
bike_data.shape

(12309, 26)

In [65]:
bike_data.isnull().sum()

participant_id             11914
accident_id                    0
case_number                11914
object_involved            11922
contributing_factor        12301
time                       12309
hour                       12309
accident_year                395
fatalities                 12309
severe_injuries            12309
emergency_room_injuries    12309
municipality                 395
police_area                  395
vehicle_type               12304
object_collided_with       11873
other_party_involved       12239
roadsegment_id               395
segment_startdate            395
segment_enddate             7667
roadnumber                   395
streetname                   395
neighborhood                 395
municipality                 395
drivingdirection             395
roadtype                     395
geometry                     395
dtype: int64

In [ ]:
#bike_data.to_csv('../data/raw/bike_data2.csv')

In [101]:
df_ams_cc = bike_data.drop(columns=['contributing_factor','roadtype', 'drivingdirection', 'case_number','object_involved','police_area', 'time', 'hour','other_party_involved','object_collided_with','emergency_room_injuries','severe_injuries','fatalities', 'vehicle_type'], errors='ignore')


In [105]:
df_ams_cc.to_csv('../data/cleaned/df_ams_cc.csv')

In [103]:
df_ams_cc

,participant_id,accident_id,accident_year,municipality,roadsegment_id,segment_startdate,segment_enddate,roadnumber,streetname,neighborhood,municipality,geometry
0,NaN,20140011930,2014.0,Amsterdam,253372001.0,2021-10-01T00:00:00,NaN,010,Ringweg-Oost,Amsterdam,Amsterdam,"LINESTRING (126588.02679999918 486233.7511, 12..."
1,NaN,20140014250,2014.0,Amsterdam,250367078.0,2021-10-01T00:00:00,NaN,010,Ringweg-Oost,Amsterdam,Amsterdam,LINESTRING (125111.05449999869 483505.16890000...
2,NaN,20140025144,2014.0,Amsterdam,258359006.0,2021-11-01T00:00:00,2022-05-31T00:00:00,N236,Provincialeweg,Amsterdam,Amsterdam,"LINESTRING (129307 479862, 129312.89799999818 ..."
3,NaN,20140030701,2014.0,Amsterdam,235372087.0,2021-10-01T00:00:00,NaN,010,Ringweg-West,Amsterdam,Amsterdam,LINESTRING (117881.25649999827 486103.59609999...
4,NaN,20140030891,2014.0,Amsterdam,252370009.0,2021-10-01T00:00:00,NaN,010,Ringweg-Oost,Amsterdam,Amsterdam,LINESTRING (126227.30389999971 485205.01480000...
...,...,...,...,...,...,...,...,...,...,...,...,...
12304,NaN,20230181631,2023.0,Amsterdam,600124746.0,2022-06-01T00:00:00,2024-05-31T00:00:00,002,Nieuwe Utrechtseweg,Amsterdam,Amsterdam,LINESTRING (122669.51669999957 482714.50039999...
12305,NaN,20230181650,2023.0,Amsterdam,600421400.0,2021-08-01T00:00:00,2024-03-31T00:00:00,200,Haarlemmerweg,Amsterdam,Amsterdam,LINESTRING (112228.52419999987 488624.11499999...
12306,NaN,20230181659,2023.0,Amsterdam,232367004.0,2021-10-01T00:00:00,NaN,004,Nieuwe Haagseweg,Amsterdam,Amsterdam,LINESTRING (116333.73530000076 483528.55849999...
12307,NaN,20230181665,2023.0,Amsterdam,600123751.0,2021-10-01T00:00:00,NaN,010,Ringweg-West,Amsterdam,Amsterdam,LINESTRING (119686.86740000173 492432.30480000...
